In [1]:
# IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
print(os.listdir())

['.ipynb_checkpoints', 'Insurance Charges - MultipleRegression.ipynb', 'insurance.csv']


# Reading file in CSV format

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
age         1338 non-null int64
sex         1338 non-null object
bmi         1338 non-null float64
children    1338 non-null int64
smoker      1338 non-null object
region      1338 non-null object
charges     1338 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.2+ KB


# the Dataset doesn't have any Null Values

# Preprocessing the dataset

In [4]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
x

array([[19, 'female', 27.9, 0, 'yes', 'southwest'],
       [18, 'male', 33.77, 1, 'no', 'southeast'],
       [28, 'male', 33.0, 3, 'no', 'southeast'],
       ...,
       [18, 'female', 36.85, 0, 'no', 'southeast'],
       [21, 'female', 25.8, 0, 'no', 'southwest'],
       [61, 'female', 29.07, 0, 'yes', 'northwest']], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
oh = OneHotEncoder()

In [6]:
x[:,1]= le.fit_transform(x[:,1])
x[:,4] = le.fit_transform(x[:,4])
x[:,5] = le.fit_transform(x[:,5])

In [7]:
x

array([[19, 0, 27.9, 0, 1, 3],
       [18, 1, 33.77, 1, 0, 2],
       [28, 1, 33.0, 3, 0, 2],
       ...,
       [18, 0, 36.85, 0, 0, 2],
       [21, 0, 25.8, 0, 0, 3],
       [61, 0, 29.07, 0, 1, 1]], dtype=object)

In [8]:
x_r= oh.fit_transform(x[:,5].reshape(-1,1)).toarray()
x_r= x_r.astype(int)
x =(x[:,:5])
x_r =(x_r[:,1:])
x = np.concatenate([x,x_r],axis=1)
x

array([[19, 0, 27.9, ..., 0, 0, 1],
       [18, 1, 33.77, ..., 0, 1, 0],
       [28, 1, 33.0, ..., 0, 1, 0],
       ...,
       [18, 0, 36.85, ..., 0, 1, 0],
       [21, 0, 25.8, ..., 0, 0, 1],
       [61, 0, 29.07, ..., 1, 0, 0]], dtype=object)

# Splitting up of Data set into Train & Test

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.3)

In [10]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

C:\Users\Mughilan_kumar\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Implementation of Multi Variate Regression

In [11]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)

In [12]:
from sklearn.metrics import r2_score
r2_score(y_pred,y_test)

0.647528322589308

In [13]:
x_train.shape

(936, 8)

# Performing Backward elimination

In [14]:
import statsmodels.formula.api as sm
x_b= np.append(np.ones((936,1)).astype(int),x_train,axis=1)
x_test = np.append(np.ones((402,1)).astype(int),x_test,axis=1)

In [15]:
x_opt = x_b[:,[0,1,2,3,4,5,6,7,8]]
regressor_ols = sm.OLS(y_train,x_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     358.9
Date:                Thu, 14 Mar 2019   Prob (F-statistic):          9.69e-278
Time:                        14:02:14   Log-Likelihood:                -9446.8
No. Observations:                 936   AIC:                         1.891e+04
Df Residuals:                     927   BIC:                         1.896e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.286e+04    192.065     66.980      0.000    1.25e+04    1.32e+04
x1          3606.0518    194.064     18.582      0.000    3225.196    3986.908
x2          -162.6387    192.966     -0.843      0.400    -541.340     216.063
x3          2144.0603    201.490     10.641      0.000    1748.631    2539.490
x4           387.0805    193.111      2.004      0.045       8.095     766.066
x5          9430.9512    193.211     48.812      0.000    9051.769    9810.134
x6          -198.5624    238.945     -0.831      0.406    -667.498     270.373
x7          -503.2317    248.028     -2.029      0.043    -989.994     -16.469
x8          -306.8770    239.601     -1.281      0.201    -777.100     163.346
==============================================================================
Omnibus:                      196.581   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              435.627
Skew:                           1.151   Prob(JB):                     2.54e-95
Kurtosis:                       5.423   Cond. No.                         2.24
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
x_opt = x_b[:,[0,1,3,4,5,6,7,8]]
regressor_ols = sm.OLS(y_train,x_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     410.2
Date:                Thu, 14 Mar 2019   Prob (F-statistic):          6.58e-279
Time:                        14:02:14   Log-Likelihood:                -9447.2
No. Observations:                 936   AIC:                         1.891e+04
Df Residuals:                     928   BIC:                         1.895e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.286e+04    192.035     66.990      0.000    1.25e+04    1.32e+04
x1          3610.2055    193.971     18.612      0.000    3229.532    3990.879
x2          2132.7746    201.013     10.610      0.000    1738.281    2527.268
x3           384.3291    193.053      1.991      0.047       5.458     763.200
x4          9422.7479    192.936     48.839      0.000    9044.106    9801.389
x5          -199.3178    238.906     -0.834      0.404    -668.176     269.540
x6          -506.6232    247.957     -2.043      0.041    -993.245     -20.002
x7          -309.6681    239.541     -1.293      0.196    -779.772     160.436
==============================================================================
Omnibus:                      198.480   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              443.701
Skew:                           1.158   Prob(JB):                     4.48e-97
Kurtosis:                       5.452   Cond. No.                         2.23
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
x_opt = x_b[:,[0,1,3,4,5,7,8]]
regressor_ols = sm.OLS(y_train,x_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     478.6
Date:                Thu, 14 Mar 2019   Prob (F-statistic):          4.08e-280
Time:                        14:02:14   Log-Likelihood:                -9447.5
No. Observations:                 936   AIC:                         1.891e+04
Df Residuals:                     929   BIC:                         1.894e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.286e+04    192.004     67.001      0.000    1.25e+04    1.32e+04
x1          3612.9994    193.911     18.632      0.000    3232.446    3993.553
x2          2134.2302    200.973     10.619      0.000    1739.817    2528.644
x3           375.8754    192.755      1.950      0.051      -2.411     754.162
x4          9424.9949    192.886     48.863      0.000    9046.453    9803.537
x5          -402.4238    214.170     -1.879      0.061    -822.737      17.890
x6          -207.3237    205.712     -1.008      0.314    -611.038     196.391
==============================================================================
Omnibus:                      196.441   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              434.182
Skew:                           1.151   Prob(JB):                     5.23e-95
Kurtosis:                       5.414   Cond. No.                         1.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
x_opt = x_b[:,[0,1,3,4,5,8]]
regressor_ols = sm.OLS(y_train,x_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     572.0
Date:                Thu, 14 Mar 2019   Prob (F-statistic):          9.42e-281
Time:                        14:02:15   Log-Likelihood:                -9449.3
No. Observations:                 936   AIC:                         1.891e+04
Df Residuals:                     930   BIC:                         1.894e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.286e+04    192.265     66.910      0.000    1.25e+04    1.32e+04
x1          3635.8225    193.793     18.761      0.000    3255.500    4016.145
x2          2029.3611    193.330     10.497      0.000    1649.947    2408.775
x3           389.4517    192.882      2.019      0.044      10.918     767.986
x4          9399.6586    192.675     48.785      0.000    9021.530    9777.787
x5           -69.2796    192.407     -0.360      0.719    -446.883     308.323
==============================================================================
Omnibus:                      198.914   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              444.096
Skew:                           1.161   Prob(JB):                     3.68e-97
Kurtosis:                       5.449   Cond. No.                         1.14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
x_opt = x_b[:,[0,1,3,4,5]]
regressor_ols = sm.OLS(y_train,x_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     715.6
Date:                Thu, 14 Mar 2019   Prob (F-statistic):          3.53e-282
Time:                        14:02:15   Log-Likelihood:                -9449.4
No. Observations:                 936   AIC:                         1.891e+04
Df Residuals:                     931   BIC:                         1.893e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.286e+04    192.175     66.942      0.000    1.25e+04    1.32e+04
x1          3634.0870    193.643     18.767      0.000    3254.061    4014.113
x2          2030.7704    193.200     10.511      0.000    1651.612    2409.929
x3           388.1603    192.758      2.014      0.044       9.869     766.452
x4          9400.5076    192.571     48.816      0.000    9022.584    9778.431
==============================================================================
Omnibus:                      199.283   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              444.917
Skew:                           1.163   Prob(JB):                     2.44e-97
Kurtosis:                       5.449   Cond. No.                         1.14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# age,bmi,children,smoker are the variables used to Predict after Backward elimination

In [20]:
x_train = x_b[:,[0,1,3,4,5]]
x_test =  x_test [:,[0,1,3,4,5]]
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)

In [21]:
r2_score(y_pred,y_test)

0.6468360993130076

In [22]:
#age,#bmi,#children,#smoker